# Homework nr. 1 - data visualization (deadline 25/10/2018)

In short, the main task is to download data on theses defended at CTU from the Internet, store them in pandas Data Frame and then visualize some hidden information.
  
> The instructions are not given in details: It is up to you to come up with ideas on how to fulfill the particular tasks as best you can. Thinking of how to visualize the data is an important part of data visualization! ;)

## What are you supposed to do:

  1. Browse the web https://dspace.cvut.cz/?locale-attribute=en and find out how to download data on Bachelor and Master theses.
  2. Download or scrape the data such that for each thesis you know the following:
    * Faculty name, department name, thesis title, thesis type (bachelor/master), supervisor name, reviewer name, year (or date) of the defence, study programme and discipline, link to a webpage with details.
  3. Store these data in one _csv_ file (should be handed in along with this notebook).
  4. Use tools available for Python to plot charts and tables to visualize/display this information:
    * Number of defended theses per year for CTU/Faculties. Distinguish the type of thesis.
    * Find the departments/study programmes/supervisors/reviewers with highest numbers of thesis and come up with some nice plots and tables to depict their numbers.
    * Mean/median/minimum/maximum number of supervised theses per year for faculties.
    * Number (or fraction) of theses supervised by people with various degrees (Bc./Ing./Ph.D./ ...).

**If you do all this properly, you will obtain 6 points**

To earn **extra two points** you can do some of these:
  * Use http://beakerx.com to make your notebook interactive in a meaningful way.
  * Come up with some other reasonable and interesting views of data.
  * Use your data to create an interactive webpage (HTML + JavaScript).

## Comments

  * Please follow the instructions from https://courses.fit.cvut.cz/MI-PDD/homeworks/index.html.
  * If the reviewing teacher is not satisfied, he can give you another chance to rework your homework and to obtain more points.

In [95]:
import numpy as np
import pandas as pd
import sklearn as skit
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup

# url with search form
urlMain = 'https://dspace.cvut.cz{}'
urlSearch = '/discover?rpp=10&etal=0&group_by=none&page={}'

pages = 40
data_all = pd.DataFrame()
page = requests.get(urlMain.format(urlSearch.format(0)))
#tree = html.fromstring(page.content)
soup = BeautifulSoup(page.text, "html.parser")
#print(soup.prettify())
for i in soup.findAll("div", {"class": "row ds-artifact-item "}):
    print ("i")
    
print(soup.findAll("div", {"class": "row ds-artifact-item "})[0].find("a").get("href"))
one = requests.get(urlMain.format(soup.findAll("div", {"class": "row ds-artifact-item "})[0].find("a").get("href")))
print(one.text)
#print(soup.findAll("div", {"class": "row ds-artifact-item "})[0].findAll("h4", {"class": "artifact-title"})[0])



i
i
i
i
i
i
i
i
i
i
/handle/10467/14194
<!DOCTYPE html>
            <!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en"> <![endif]-->
            <!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8" lang="en"> <![endif]-->
            <!--[if IE 8]>    <html class="no-js lt-ie9" lang="en"> <![endif]-->
            <!--[if gt IE 8]><!--> <html class="no-js" lang="en"> <!--<![endif]-->
            <head><META http-equiv="Content-Type" content="text/html; charset=UTF-8">
<meta content="text/html; charset=UTF-8" http-equiv="Content-Type">
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible">
<meta content="width=device-width,initial-scale=1" name="viewport">
<link rel="shortcut icon" href="/themes/CVUT/images/favicon.ico">
<link rel="apple-touch-icon" href="/themes/CVUT/images/apple-touch-icon.png">
<meta name="Generator" content="DSpace 5.5">
<link href="/themes/CVUT/styles/main.css" rel="stylesheet">
<link type="application/opensearchdescription+xml" rel="search"